**k-Means clustering and cluster analysis**

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

/home/parth/.local/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
stockData = pd.read_csv('https://raw.githubusercontent.com/djcrow-instructor/datasets/main/StockDataWithoutClusters_v2.csv')    

In [3]:
stockData.head()

Company            Sector  Price  \
0  Hindustan Oil Exploration Company Ltd.         Crude Oil  68.40   
1        United Breweries (Holdings) Ltd.           Alcohol  40.00   
2                   Hindustan Motors Ltd.  Auto & Auto Anc.   7.15   
3                                HMT Ltd.       Industrials  42.85   
4                       ABG Shipyard Ltd.       Industrials  31.10   

   Market Cap  Free Float Market Cap %    6m ADV     RoE %    RoCE %  \
0  131.299505                   0.8371  1.136947 -1.000000 -0.046311   
1   39.316572                   0.4766  0.162805 -1.000000 -0.535856   
2   21.946366                   0.6766  0.041085 -0.868864 -1.000000   
3  477.697516                   0.0631  0.013322 -0.183166 -0.167937   
4   24.718436                   0.8177  0.178504 -0.766749 -0.051679   

   EBIT Margin %         EPS  PAT %  Stock Return %  
0      -0.923766    4.533827   -1.0       90.794979  
1      -1.000000  158.744001   -1.0       38.169257  
2      -1.000000    6.880137   -1.0       -8.333333  
3      -1.000000    5.865182   -1.0      -23.070018  
4      -0.865160  182.175768   -1.0      -57.014513

In [ ]:
# All % columns were on a scale where 1 meant 100%, except for Stock Return %. We multiply all % columns (except Returns) by 100
stockData['Free Float Market Cap %'] = stockData['Free Float Market Cap %']*100
stockData['RoE %'] = stockData['RoE %']*100
stockData['RoCE %'] = stockData['RoCE %']*100
stockData['EBIT Margin %'] = stockData['EBIT Margin %']*100
stockData['PAT %'] = stockData['PAT %']*100
stockData.describe()

Price    Market Cap  ...        PAT %  Stock Return %
count   1225.000000   1225.000000  ...  1225.000000     1225.000000
mean     524.523282   1007.097584  ...     3.048752        7.087798
std     1975.921838   3977.721963  ...    17.421837       50.988015
min        0.350000      1.027453  ...  -100.000000      -91.128010
25%       60.550000     40.744259  ...     0.654241      -21.670429
50%      170.100000    113.898556  ...     3.935423       -1.576713
75%      448.050000    417.201805  ...     8.648472       25.083403
max    51579.200000  66127.302990  ...   100.000000      586.043761

[8 rows x 10 columns]

In [ ]:
features = stockData.columns[2:]
features

Index(['Price', 'Market Cap', 'Free Float Market Cap %', '6m ADV', 'RoE %',
       'RoCE %', 'EBIT Margin %', 'EPS', 'PAT %', 'Stock Return %'],
      dtype='object')

In [ ]:
# Good idea to standardize the features before k-Means
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
stockDataFeatures_scaled = scaler.fit_transform(stockData[features])
stockDataFeatures_scaled = pd.DataFrame(stockDataFeatures_scaled, columns=features)
stockDataFeatures_scaled.describe()

Price   Market Cap  ...        PAT %  Stock Return %
count  1225.000000  1225.000000  ...  1225.000000     1225.000000
mean      0.010163     0.015214  ...     0.515244        0.145038
std       0.038309     0.060153  ...     0.087109        0.075296
min       0.000000     0.000000  ...     0.000000        0.000000
25%       0.001167     0.000601  ...     0.503271        0.102570
50%       0.003291     0.001707  ...     0.519677        0.132243
75%       0.008680     0.006294  ...     0.543242        0.171613
max       1.000000     1.000000  ...     1.000000        1.000000

[8 rows x 10 columns]

In [ ]:
kmeans = KMeans(7, n_jobs=-1)

In [ ]:
clus = kmeans.fit_predict(stockDataFeatures_scaled)

In [ ]:
stockData['cluster'] = clus
stockData['cluster'].value_counts()

1    432
5    404
2    168
0    124
3     63
6     19
4     15
Name: cluster, dtype: int64

In [ ]:
clusterDesc = pd.DataFrame(stockData.iloc[:,2:].groupby('cluster').mean().round(3))
clusterDesc.insert(0,'size',stockData['cluster'].value_counts())

In [ ]:
clusterDesc

size     Price  Market Cap  ...      EPS   PAT %  Stock Return %
cluster                              ...                                 
0         124  1080.754    2705.938  ...   34.712  17.210           7.621
1         432   598.294     569.398  ...  103.236   5.801           8.996
2         168   630.690     635.866  ...  208.097   3.181           2.828
3          63    92.130     218.797  ...  308.978 -23.213          -9.171
4          15  1013.843   25916.713  ...  102.046   8.390          37.485
5         404   303.213     349.926  ...  152.188   3.605           9.032
6          19    31.530      76.196  ...   90.791 -82.077         -13.529

[7 rows x 11 columns]